In [73]:
import pandas as pd
import os

repo_path = f"""{os.getcwd().split("jobseeker-analytics")[0]}jobseeker-analytics"""
labeled_df = pd.read_json(f"{repo_path}/backend/training_data.json", convert_dates=False)[["id", "application_status"]]
gemini_df = pd.read_csv(f"{repo_path}/backend/data_gemini_llm.csv", header=0)[["Message ID", "Application Status"]]

accuracy_df = labeled_df.merge(gemini_df, left_on="id", right_on="Message ID", how="inner")
unique_message_ids = accuracy_df.nunique().id
unique_gemini_ids = gemini_df.nunique()["Message ID"]
unique_labeled_ids = labeled_df.nunique().id

accuracy_df["correct"] = accuracy_df["application_status"] == accuracy_df["Application Status"]
accuracy_counts = accuracy_df["correct"].value_counts().reset_index()
correct_counts = accuracy_df["correct"].value_counts().reset_index().loc[0, "count"]
print(correct_counts)
total_counts = accuracy_df.shape[0]
print(total_counts)
print(f"accuracy is {correct_counts / total_counts * 100:.2f}%")

# accuracy within each category
accuracy_by_category = accuracy_df.groupby("application_status")["correct"].value_counts().unstack().fillna(0)
accuracy_by_category["accuracy"] = accuracy_by_category[True] / (accuracy_by_category[True] + accuracy_by_category[False])
accuracy_by_category = accuracy_by_category.reset_index()
accuracy_by_category = accuracy_by_category.rename(columns={"application_status": "Application Status"})
accuracy_by_category = accuracy_by_category[["Application Status", True, False, "accuracy"]]
print(accuracy_by_category)

393
524
accuracy is 75.00%
correct                         Application Status   True  False  accuracy
0                     Action required from company    5.0   25.0  0.166667
1                         Application confirmation  156.0   10.0  0.939759
2                                  Assessment sent    8.0    1.0  0.888889
3                             Availability request   21.0   55.0  0.276316
4                  Did not apply - inbound request    9.0    0.0  1.000000
5        False positive, not related to job search    0.0   21.0  0.000000
6                       Hiring freeze notification    1.0    0.0  1.000000
7                              Information request    5.0   16.0  0.238095
8                             Interview invitation  120.0    3.0  0.975610
9                                       Offer made    1.0    0.0  1.000000
10                                       Rejection   65.0    0.0  1.000000
11                            Withdrew application    2.0    0.0  1.00000